In [2]:
from mutwo import core_events
from mutwo import midi_converters
from mutwo import music_parameters

In [3]:
class Note(core_events.SimpleEvent):
    # Dynamik ist konstant
    volume = music_parameters.WesternVolume('p')

    def __init__(self, pitch, duration):
        self.pitch = pitch
        
        # Das ist Python spezifischer Syntax, um dem Konstruktor
        # der Basisklasse das "duration" Argument zu übermitteln.
        super().__init__(duration)

In [4]:
melody = core_events.SequentialEvent(
    [
        Note(music_parameters.WesternPitch('c'), 1),
        Note(music_parameters.WesternPitch('d'), 1),
    ]
)

In [5]:
# Definiere zuerst den Übersetzer
event_to_midi_file = midi_converters.EventToMidiFile(
    simple_event_to_pitch_list=lambda simple_event: [
        getattr(simple_event, 'pitch')
    ]
)
# Übersetze jetzt die Melodie
event_to_midi_file.convert(melody, 'my_melody.mid')

[[WesternPitch('c', 4)], WesternVolume(p), ()]
(Message('pitchwheel', channel=0, pitch=0, time=0), Message('note_on', channel=0, note=60, velocity=46, time=0), Message('note_off', channel=0, note=60, velocity=46, time=480))
[[WesternPitch('d', 4)], WesternVolume(p), ()]
(Message('pitchwheel', channel=1, pitch=0, time=479), Message('note_on', channel=1, note=62, velocity=46, time=480), Message('note_off', channel=1, note=62, velocity=46, time=960))


/home/levinericzimmermann/Texts/21BAmutwo/jupyter/.venv/lib/python3.9/site-packages/mutwo/core_events/envelopes.py:382: InvalidAverageValueStartAndEndWarning: Average value for range where start == end is equal to envelope.value_at(start)!
  warnings.warn(core_utilities.InvalidAverageValueStartAndEndWarning())


# Unspezifische Ereignisse

## Dynamische Attribute

Dieses Jupyter Notebook demonstriert den Umgang mit dynamischen Attributen.

Die MIDI Datei wurde im vorgehenden Beispiel erfolgreich erzeugt. Die erzeugte MIDI Datei enthält *Note On* und *Note Off* Nachrichten, denen eine bestimmte *velocity* zugeordnet ist. Allerdings können MIDI Dateien auch noch weitere Nachrichten enthalten, wie z. B. Kontrollnachrichten. In der API Dokumentation der Klasse *EventToMidiFile* kann das Argument *simple_event_to_control_message_tuple* gefunden werden.

Ist jetzt intendiert, dass die MIDI Datei auch Kontrollnachrichten (z. B. zur Steuerung der Klangsynthese) enthält, könnte eine neue *Note* Klasse definiert werden. Aber vielleicht brauchen die meisten Noteninstanzen keine Kontrollwerte. In dem Fall mag es eleganter sein, bestimmten Noten in der Melodie dynamisch Kontrollnachrichten hinzuzufügen.

In [6]:
# Importiere das externe mido Paket, um Kontrollnachrichten
# initalisieren zu können. Mutwo verwendet auch mido um MIDI
# Dateien zu lesen und zu schreiben.
import mido
# Ordne jetzt der ersten Note Kontrollwerte zu.
# Siehe die mido Dokumentation bezüglich der Initialisierung
# von Nachrichteninstanzen.
melody[0].control_message_tuple = (
    mido.Message("control_change", channel=0, control=10, value=127),
    mido.Message("control_change", channel=0, control=11, value=64),
)
# Übersetze jetzt die Melodie mit den Kontrollnachrichten.
event_to_midi_file.convert(melody, 'my_melody.mid')

[[WesternPitch('c', 4)], WesternVolume(p), (Message('control_change', channel=0, control=10, value=127, time=0), Message('control_change', channel=0, control=11, value=64, time=0))]
(Message('control_change', channel=0, control=10, value=127, time=0), Message('control_change', channel=0, control=11, value=64, time=0), Message('pitchwheel', channel=0, pitch=0, time=0), Message('note_on', channel=0, note=60, velocity=46, time=0), Message('note_off', channel=0, note=60, velocity=46, time=480))
[[WesternPitch('d', 4)], WesternVolume(p), ()]
(Message('pitchwheel', channel=1, pitch=0, time=479), Message('note_on', channel=1, note=62, velocity=46, time=480), Message('note_off', channel=1, note=62, velocity=46, time=960))
